In [55]:
import os
import pandas as pd
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import datetime

In [62]:
class init_loop_vrp:
    def __init__(self):
        pass
        
    def print_solution(self):
        """Prints solution on console."""
        print('Loop route 구축')
        self.all_loop_route = []
        self.demand_route = []
        for vehicle_id in range(self.data['num_vehicles']):
            self.vehicle_route = []
            index = self.routing.Start(vehicle_id)

            while not self.routing.IsEnd(index):
                node_index = self.manager.IndexToNode(index)
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                self.vehicle_route.append(node_index)

            self.all_loop_route.append(self.vehicle_route)
          
        for route in self.all_loop_route:
            self.car_demand = []
            for i in route:
                self.car_demand.append(self.data['demands'][i])

            self.demand_route.append(self.car_demand)

    def get_loop(self, input_node, time_matrix, num_vehicles=2):
        '''
        num_vehicles : 사용할 차량 대수 (int) = 2대로 고정
        input_node : 초기에 사용자가 신청한 노드의 index (list)
        time_matrix : 가능한 모든 노드 간의 time matrix (array)
        '''
        """Entry point of the program."""
        # Instantiate the data problem.
        self.data = {}
        self.data['num_vehicles'] = num_vehicles
        self.data['time_matrix'] = pd.DataFrame(time_matrix)[input_node].iloc[input_node].to_numpy()
        self.data['vehicle_capacities'] = [6 for _ in range(num_vehicles)]
        #print(self.data['vehicle_capacities'])
        self.data['demands'] = [1 for i in range(len(input_node))]  #MVP 테스트를 위해 수정 #[0] + [sum(self.data['vehicle_capacities'])-len(time_matrix)] + [1 for i in range(len(time_matrix))] 
        self.data['depot'] = 0

        # Create the routing index manager.
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['time_matrix']), self.data['num_vehicles'], self.data['depot'])

        # Create Routing Model.
        self.routing = pywrapcp.RoutingModel(self.manager)


        # 1-1. Define cost of each arc.
        def distance_callback(from_index, to_index):
            """Returns the manhattan distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.data['time_matrix'][from_node][to_node]

        transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # 1-2.Add Distance constraint.
        dimension_name = 'Distance'
        self.routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            30000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)

        distance_dimension = self.routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # 2-1.Define cost of each node.
        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            return self.data['demands'][from_node]

        demand_callback_index = self.routing.RegisterUnaryTransitCallback(
            demand_callback)

        # 2-2.Add Capacity constraint.
        self.routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            self.data['vehicle_capacities'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity')

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

        # Solve the problem.
        self.solution = self.routing.SolveWithParameters(search_parameters)
    

        # Print solution on console.
        if self.solution:
            self.print_solution()
            return self.all_loop_route, self.demand_route
        else:
            print(self.solution)

In [63]:
class straight_route:
    def __init__(self):
        pass

    def print_solution(self):
        print('\n')
        """Prints solution on console."""
        print(f'Objective: {self.solution.ObjectiveValue()}')
        total_time = 0
        total_load = 0

        for vehicle_id in range(self.data['num_vehicles']):
            self.straight_route = []
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
            route_time = 0
            route_time_lst = [0]
            route_load = 0
            i=0

            while not self.routing.IsEnd(index):
                node_index = self.manager.IndexToNode(index)
                route_load += self.data['demands'][node_index]
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                route_time_lst.append(self.routing.GetArcCostForVehicle(previous_index, index, vehicle_id))
                route_time += self.routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

                plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(self.data['name_lst'][self.input_node[node_index]], self.data['vehicle_capacities'][vehicle_id] - route_load, self.data['demands'][node_index], datetime.timedelta(seconds = route_time_lst[i]))
                i+=1
                
                self.straight_route.append(self.input_node[node_index])

            node_index = self.manager.IndexToNode(index)
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(self.data['name_lst'][self.input_node[node_index]], self.data['vehicle_capacities'][vehicle_id] - route_load, self.data['demands'][node_index], datetime.timedelta(seconds = route_time_lst[i]))
            plan_output += '서비스 종료\n'
            self.straight_route.append(self.input_node[node_index])

            print(plan_output)

            total_time += route_time
            total_load += route_load

        print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
        print('Total load of all routes: {}'.format(total_load))


        return self.straight_route
   

    def get_straight_route(self, time_matrix, input_node, name_lst, demand):
        """Entry point of the program."""
        self.input_node = input_node
        self.data = {}
        self.data['name_lst'] = name_lst
        self.data['new_time_matrix'] = pd.DataFrame(time_matrix)[self.input_node].iloc[self.input_node].to_numpy()
        self.data['demands'] = demand
        self.data['vehicle_capacities'] = [4]
        self.data['num_vehicles'] = 1
        self.data['starts'] = [0]
        self.data['ends'] = [int(np.argmax(self.data['new_time_matrix'][0]))]
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['new_time_matrix']), self.data['num_vehicles'], self.data['starts'], self.data['ends'])

        # Create Routing Model.
        self.routing = pywrapcp.RoutingModel(self.manager)


        # 1-1. Define cost of each arc.
        def distance_callback(from_index, to_index):
            """Returns the manhattan distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.data['new_time_matrix'][from_node][to_node]

        transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # 1-2.Add Distance constraint.
        dimension_name = 'Distance'
        self.routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            8000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = self.routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # 2-1.Define cost of each node.
        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            return self.data['demands'][from_node]

        demand_callback_index = self.routing.RegisterUnaryTransitCallback(
            demand_callback)

        # 2-2.Add Capacity constraint.
        self.routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            self.data['vehicle_capacities'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity')

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

        # Solve the problem.
        self.solution = self.routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        self.name = []
        if self.solution:
            self.print_solution()
            for i in self.straight_route:
                self.name.append(self.data['name_lst'][i])
                
            return self.straight_route, self.name
            
        else:
          print(self.solution)



In [64]:
class WTB:
    def __init__(self, time_matrix, name_lst, num_vehicles=2):
        self.num_vehicles = num_vehicles
        self.time_matrix = time_matrix
        self.name_lst = name_lst #초기에 들어오는 노드


    def get_route(self, input_node):
        self.input_node = input_node
        # Loop route
        self.loop_initial = init_loop_vrp()
        self.initial_loop_result= self.loop_initial.get_loop(self.input_node, self.time_matrix, self.num_vehicles) 

        self.loop = self.initial_loop_result[0] # 차량 별 loop구조로 생기는 route
        self.demand = self.initial_loop_result[1] # 차량 별 demand

        # Straight route
        self.loop_straight = []
        loop2st = straight_route()
        
        for i in range(len(self.loop)):
            self.straight =  loop2st.get_straight_route(self.time_matrix, self.loop[i], self.name_lst, self.demand[i])
            self.loop_straight.append(self.straight[0])   


In [69]:
time_matrix = np.load('고연전_MVP_테스트_경로매트릭스(분).npy')
time_matrix

array([[   0,  907,  849, 1003,  755, 1225, 1348],
       [1072,    0,  372,  521,  754,  743,  843],
       [1131,   59,    0,  580,  812,  802,  902],
       [1254,  651,  705,    0, 1086,  475,  471],
       [1025,  153,   94,  674,    0,  896,  996],
       [1872, 1048, 1102,  868, 1483,    0,  362],
       [1510,  930,  984,  506, 1365,  228,    0]])

In [66]:
input_node = [1,2,4]
pd.DataFrame(time_matrix)[input_node].iloc[input_node].to_numpy()

array([[  0, 372, 754],
       [ 59,   0, 812],
       [153,  94,   0]])

In [67]:
'''
입력값
1. bus_name_lst : 버스 정류장 명 입력 (리스트 형식)
2. get_route : 가야하는 버스 정류장 인덱스 번호 입력 (리스트 형식)
'''

bus_name_lst = ['우신향병원주차장','이문동교회','여의도역','홍대입구역','동천학교','63빌딩,가톨릭대학교여의도성모병원','태릉입구역7번출구.서울생활사박물관','성원경남아너스빌앞']

gangnam = WTB( time_matrix, bus_name_lst, num_vehicles=2)
gangnam.get_route([0:8]) # ( 수요 버스정류장 노드 번호 입력하기 : 총 8개 )

Loop route 구축


Objective: 91708
Route for vehicle 0:

 정류장 이름 : 우신향병원주차장  (탑승객 수: 3, 하차수: 1, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 홍대입구역  (탑승객 수: 2, 하차수: 1, 소요시간: 0:14:09) 
    ==>     정류장 이름 : 이문동교회  (탑승객 수: 2, 하차수: 1, 소요시간: 0:00:59) 
    ==>    서비스 종료

Total Duration of all routes: 0:15:08m
Total load of all routes: 2


Objective: 0
Route for vehicle 0:

 정류장 이름 : 우신향병원주차장  (탑승객 수: 3, 하차수: 1, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 동천학교  (탑승객 수: 3, 하차수: 1, 소요시간: 0:00:00) 
    ==>    서비스 종료

Total Duration of all routes: 0:00:00m
Total load of all routes: 1
